In [3]:
import numpy as np
from mne.decoding import CSP
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from util2 import A3Load
DIR, loc, EEG_eval, EEGL_eval, EEGR_eval, EEGL_train, EEGR_train = A3Load()

In [20]:
# need to format the data for MNE, ugh
I_PARTICIPANT = 6
I_FILTER = 1

N_EPOCHS = 72
N_CHANNELS = 22

l_train = []
r_train = []
for i in range(N_EPOCHS):
    l_train.append(EEGL_train[I_PARTICIPANT][I_FILTER][:,:,i])
    r_train.append(EEGR_train[I_PARTICIPANT][I_FILTER][:,:,i])

X = np.array(l_train + r_train)
X = X.astype(np.float64)

y_l = np.zeros(len(l_train))
y_r = np.ones(len(r_train))

y = np.concatenate((y_l, y_r))

X.shape, y.shape

((144, 22, 550), (144,))

In [17]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CSP
csp = CSP(n_components=4, norm_trace=False)

# Fit CSP
csp.fit(X, y)

Computing rank from data with rank=None
    Using tolerance 15 (2.2e-16 eps * 22 dim * 3.1e+15  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None


    Using tolerance 16 (2.2e-16 eps * 22 dim * 3.2e+15  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.


CSP({'component_order': 'mutual_info',
 'cov_est': 'concat',
 'cov_method_params': None,
 'log': None,
 'n_components': 4,
 'norm_trace': False,
 'rank': None,
 'reg': None,
 'transform_into': 'average_power'})

In [21]:
'''
Right/Left classifier
'''
# CSP transformattion on training data

X_filt_train = csp.transform(X_train)
print(X_filt_train.shape)

# Setting up LDA for training
LDA = LinearDiscriminantAnalysis()

LDA.fit(X_filt_train, y_train)


# CSP transformattion on testing data
X_filt_test = csp.transform(X_test)

#predictions = LDA.predict(X_filt_test)
#acc = LDA.score(X_filt_test, y_test)
predictions = LDA.predict(X_filt_test)
acc = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f"accuracy: {acc} \n precision: {precision} \n recall: {recall} \n f1: {f1}")

(115, 4)
accuracy: 0.9655172413793104 
 precision: 0.9285714285714286 
 recall: 1.0 
 f1: 0.962962962962963


In [ ]:
'''
Right/Baseline
'''